### Table of Contents

* [Librairies and setup](#1)
* [Read data](#2)
* [Data Exploration](#3)
* [Duplicates](#4)
    * [Bids to Impression duplicates](#4.1)
    * [Impressions to clicks duplicates](#4.2)
* [Join full data set](#5)
    * [Reduce dataset size](#5.0)
    * [Sanity Check before joining](#5.0.1)
    * [Join all](#5.1)
    * [Sample the data](#5.2)
* [Add user profile info](#6)

# Librairies and setup <a class="anchor" id="1"></a>

In [173]:
#pip install os
import os, glob, bz2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [174]:
os.getcwd()

'/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd'

In [175]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [176]:
os.chdir('/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd')

In [161]:
import zipfile,glob
for filename in glob.glob("*.bz2"):
    print(os.getcwd()+'/'+filename)
    with zipfile.ZipFile(os.getcwd()+'/'+filename, 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())

/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd/clk.20130612.txt.bz2


BadZipFile: File is not a zip file

In [162]:
glob.glob("*.bz2")

['clk.20130612.txt.bz2',
 'conv.20130606.txt.bz2',
 'bid.20130606.txt.bz2',
 'bid.20130607.txt.bz2',
 'conv.20130607.txt.bz2',
 'clk.20130608.txt.bz2',
 'clk.20130611.txt.bz2',
 'imp.20130606.txt.bz2',
 'clk.20130610.txt.bz2',
 'imp.20130607.txt.bz2',
 'clk.20130609.txt.bz2',
 'bid.20130611.txt.bz2',
 'conv.20130608.txt.bz2',
 'bid.20130608.txt.bz2',
 'imp.20130612.txt.bz2',
 'conv.20130611.txt.bz2',
 'conv.20130610.txt.bz2',
 'bid.20130609.txt.bz2',
 'conv.20130609.txt.bz2',
 'bid.20130610.txt.bz2',
 'conv.20130612.txt.bz2',
 'imp.20130611.txt.bz2',
 'clk.20130606.txt.bz2',
 'imp.20130608.txt.bz2',
 'bid.20130612.txt.bz2',
 'imp.20130609.txt.bz2',
 'imp.20130610.txt.bz2',
 'clk.20130607.txt.bz2']

In [180]:
dates_list=['20130606'
           , '20130607'
           , '20130608'
           , '20130609'
           , '20130610'
           , '20130611'
           , '20130612']
for date in dates_list:
    print(date)
    unzipped_file = bz2.BZ2File('bid.'+date+'.txt.bz2', "r")
    bids=pd.read_table(unzipped_file, names=header_season2_bids, nrows=10)
    
    unzipped_file = bz2.BZ2File('imp.'+date+'.txt.bz2', "r")
    impressions=pd.read_table(unzipped_file, names=header_season2, nrows=10)
    
    unzipped_file = bz2.BZ2File('clk.'+date+'.txt.bz2', "r")
    clicks=pd.read_table(unzipped_file, names=header_season2, nrows=10)
    
    unzipped_file = bz2.BZ2File('conv.'+date+'.txt.bz2', "r")
    conversions=pd.read_table(unzipped_file, names=header_season2, nrows=10)


20130606
20130607
20130608
20130609
20130610
20130611
20130612


In [178]:
conversions

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
0,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000132000,3,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN
1,eafdc6440e4da4b74edacb8eb4f075ac,20130606000259000,3,VhKOZAMi3UsHQb2,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",60.212.0.*,146,152,1,trqRTvpogNlyDok4JKTI,4eeb6114aabd43509c51b4b6aa28422a,NaN,mm_10032051_2374052_9577342,950,90,2,1,0,d5cecca9a6cbd7a0a48110f1306b26d1,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,"10063,10006,13866,10111"
2,7650fc7765a02637c2ad5ee26e3d1e41,20130606005000000,3,Vhk7ZpR4O8T6BOB,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; Microsoft Windows Media Center PC 6.0; Media Center PC 6.0),60.195.3.*,1,1,2,DFpETuMJM95Y1mKYUV,b7cb3e22216e965433761ac7fba4a2ba,NaN,3043027207,200,200,2,0,5,5c4e0bb0db45e2d1b3a14f817196ebd6,238,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN
3,3f398ab94ec3bf13f359e5ef5f050f83,20130606023027000,3,VhkrP3x3OQp6wsc,"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Maxthon/4.0.6.2000 Chrome/26.0.1410.43 Safari/537.1",222.69.40.*,79,79,2,trqRTuFJBT27Fgc,63e07945a1fcacbd54e7d8d2ecc0b3af,NaN,2640582,300,250,0,0,5,d881a6c788e76c2c27ed1ef04f119544,238,0,d29e59bf0f7f8243858b8183f14d4412,3358,"10024,13800,10110"
4,4893adaad55ba10776fe85c438de2bcb,20130606065018000,3,Vh20L5B7DtjflVj,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",106.87.180.*,275,275,1,trqRTvuJX9q7kTKbuKz,4c2a8aa32aec900e59477a25351f5fed,NaN,mm_11226918_3459901_11252208,300,250,1,5,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,"13403,10031,10075,13866,10063,10024,10057,10079,10083,10067,10006,10059,10048,10111"
5,44a998f731d11369c5b9d43fbd16fb2d,20130606065143000,3,Vhk7VuTLLHFHkMz,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0),116.232.232.*,79,79,1,tFKETuqyMo1mjMp45SqfNX,249b2c34247d400ef1cd3c6bfda4f12a,NaN,mm_11402872_1272384_3182279,300,250,1,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN
6,fc5e4b5857b30afee9df81f564905337,20130606070221000,3,Vh1LL6d2P6pdGWB,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",222.140.141.*,164,179,1,trqRTvuaX9C7gspy,d647d5bdf8d4fc11a081f943a0894d77,NaN,mm_28015408_3482389_11361056,300,250,0,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,"10006,10110"
7,fc5e4b5857b30afee9df81f564905337,20130606070221000,3,Vh1LL6d2P6pdGWB,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",222.140.141.*,164,179,1,trqRTvuaX9C7gspy,d647d5bdf8d4fc11a081f943a0894d77,NaN,mm_28015408_3482389_11361056,300,250,0,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,"10006,10110"
8,ec8d7ce73e46ae9640dfd7a6c0be0b7a,20130606094420000,3,Vhk7C5F6P9ff3-f,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; .NET CLR 1.1.4322; .NET CLR 2.0.50727),180.166.78.*,79,79,1,DSmreF9aBTN7gqKbuKz,96e0f84e00139192e4ef57a797e3353,NaN,mm_13181159_1847728_9696846,950,90,1,1,0,d5cecca9a6cbd7a0a48110f1306b26d1,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,"10048,10063,10006"
9,1228435e8d54a1f239108ae9a90ed3ed,20130606095544000,3,Vh1DLipvP2mdtOB,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0),119.123.205.*,216,219,2,trqRTup7Q9u

# Read data <a class="anchor" id="2"></a>

In [5]:
header_season2_bids=[
    'Bid ID'
    ,'Timestamp'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Advertiser ID'
    ,'User Profile IDs']

header_season2=[
    'Bid ID'
    ,'Timestamp'
    ,'Log Type'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Paying Price'
    ,'Landing Page URL'
    ,'Advertiser ID'
    ,'User Profile IDs']

bids=pd.read_table('bid.20130606.txt', names=header_season2_bids)
impressions=pd.read_table('imp.20130606.txt', names=header_season2)
clicks=pd.read_table('clk.20130606.txt', names=header_season2)
conversions=pd.read_table('conv.20130606.txt', names=header_season2)

all_data={'bids': bids, 'impressions':impressions, 'clicks':clicks, 'conversions':conversions}
for key, value in all_data.items(): 
    print('shape of %s is %s'%(key, value.shape))

/Users/Armand/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


shape of bids is (9586949, 21)
shape of impressions is (1821350, 24)
shape of clicks is (1289, 24)
shape of conversions is (43, 24)


# Data Exploration  <a class="anchor" id="3"></a>

Let's see if a record in conversions exists in clicks, impressions and bids

In [6]:
conversions.head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
0,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000132000,3,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


In [7]:
clicks[clicks['Bid ID']=='690e543c46a6cbe8bdd062f53a8ac7a2']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
8,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000125692,2,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,0,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


In [8]:
impressions[impressions['Bid ID']=='690e543c46a6cbe8bdd062f53a8ac7a2']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
15662,690e543c46a6cbe8bdd062f53a8ac7a2,20130606000111948,1,Vhk7ZApiDvTVtMn,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),59.60.86.*,124,133,1,3FF-e59aG5s0JmE4JKTIqxM_,5d6a0e40983111ee4209640ef1ae601d,NaN,mm_33726324_3429225_11100772,300,250,255,1,0,d881a6c788e76c2c27ed1ef04f119544,227,71,d29e59bf0f7f8243858b8183f14d4412,3358,NaN


Cool ! looks like the logic is OK

# Duplicates  <a class="anchor" id="4"></a> 

In [9]:
bids['Bid ID'].duplicated().sum()

0

No duplicated Bids based on 'Bid ID' which is good news! 

## Bids --> Impression duplicates <a class="anchor" id="4.1"></a> 

When trying to join the data, we see that impression bid IDs are not unique. Let's see which ones are duplicated

In [10]:
impressions['Bid ID'].duplicated().sum()

6275

... let's find the records assciated with these 

In [11]:
impressions[impressions['Bid ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
249,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105378,1,Vhk7ZAnyPQq2Xb2,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN


In [12]:
impressions[impressions['Bid ID']=='8678cd6a8dc3c8cda78a332db4e65e26']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
199,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105295,1,Vh1T1uqVL6Mfwgc,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN
249,8678cd6a8dc3c8cda78a332db4e65e26,20130606000105378,1,Vhk7ZAnyPQq2Xb2,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.22 (KHTML, like Gecko) Chrome/25.0.1364.84 Safari/537.22 BIDUPlayerBrowser/2.x",112.85.159.*,80,86,1,trqRTvMEB9T71m58uG,50449e3fa8bbee1fb2477f8483176bec,NaN,mm_10032134_3502082_11459720,300,250,0,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,187,361e128affece850342293213691a043,3386,NaN


different ipinyou id --> maybe ipinyouid is unique 

In [13]:
impressions[impressions['iPinYou ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
94,4571654093fdb94a3d2b3485e159e51e,20130606000105011,1,VhkSO71iPlM6FVc,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",175.1.175.*,201,204,2,trqRTvuYMZk7FmMs,94bee0f33b49b1c4b83906775196f4ec,NaN,86730593,300,250,1,0,5,d881a6c788e76c2c27ed1ef04f119544,238,166,d29e59bf0f7f8243858b8183f14d4412,3358,"10063,13866,10006,13800,10093,10111"


Nope, iPinYou ID is not unique. Maybe the combination of IpinYouID + Bid ID ???

In [14]:
impressions[impressions[['Bid ID','iPinYou ID']].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
5162,59088cb88db642750119b5d9eea721d7,20130606000107572,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"


In [15]:
impressions[impressions['Bid ID']=='59088cb88db642750119b5d9eea721d7']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4501,59088cb88db642750119b5d9eea721d7,20130606000107300,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"
5162,59088cb88db642750119b5d9eea721d7,20130606000107572,1,Vh1dOap7DcNcto2,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 718),221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,"10006,10083,10110"
62824,59088cb88db642750119b5d9eea721d7,20130606003107393,1,Vhk7Znl8PvNPD-C,Mozilla/4.0,221.206.167.*,65,75,1,trqRTvuvXZFR1m58uG,c65ebed2c78716a23561128bb9c00a99,NaN,mm_29512587_2494707_11117724,160,600,2,1,0,80a776343079ed94d424f4607b35fd39,238,64,aa4976c67db0f076c563a29f6697928b,3476,10063


conclusion 1 to many relationship between bids and impressions

## Impressions to clicks duplicates <a class="anchor" id="4.2"></a> 

In [16]:
clicks[clicks['Bid ID'].duplicated()].head(1)

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
6,47e7c5523cbda762a7c00e0c47c41993,20130606000123750,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


In [17]:
clicks[clicks['Bid ID']=='47e7c5523cbda762a7c00e0c47c41993']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4,47e7c5523cbda762a7c00e0c47c41993,20130606000120463,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN
6,47e7c5523cbda762a7c00e0c47c41993,20130606000123750,2,Vhk7ZApCP4BWjgk,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,0,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


In [18]:

impressions[impressions['Bid ID']=='47e7c5523cbda762a7c00e0c47c41993']

,Bid ID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region ID,City ID,Ad Exchange,Domain,URL,Anonymous URL,Ad Slot ID,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Creative ID,Bidding Price,Paying Price,Landing Page URL,Advertiser ID,User Profile IDs
4056,47e7c5523cbda762a7c00e0c47c41993,20130606000107128,1,Vhk7ZAn8OvpPMI2,"MQQBrowser/43 Mozilla/5.0 (iPhone 5ATT; CPU iPhone OS 6_1 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Mobile/10B143 Safari/7534.48.3",14.212.204.*,216,222,1,erdbexb0gI5RgsxfU11_qxMzmw,ff1cdca6dda6054ef6ace9442ace3a20,NaN,mm_12581624_3138783_11005213,300,250,1,5,0,00fccc64a1ee2809348509b7ac2a97a5,227,113,b2e35064f3549d447edbbdfb1f707c8c,3427,NaN


Conclusion --> 1 to many relationship between impressions and clicks 

# Left join to create full dataset <a class='anchor' id="5"></a>

## Reduce data set size (drop and dummify) <a class='anchor' id="5.0"></a>

Let's first reduce the dataset size by dropping irrelevant columns

In [19]:
print(bids.columns)
print(impressions.columns)
print(clicks.columns)

Index(['Bid ID', 'Timestamp', 'iPinYou ID', 'User-Agent', 'IP', 'Region ID',
       'City ID', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL',
       'Ad Slot ID', 'Ad Slot Width', 'Ad Slot Height', 'Ad Slot Visibility',
       'Ad Slot Format', 'Ad Slot Floor Price', 'Creative ID', 'Bidding Price',
       'Advertiser ID', 'User Profile IDs'],
      dtype='object')
Index(['Bid ID', 'Timestamp', 'Log Type', 'iPinYou ID', 'User-Agent', 'IP',
       'Region ID', 'City ID', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL',
       'Ad Slot ID', 'Ad Slot Width', 'Ad Slot Height', 'Ad Slot Visibility',
       'Ad Slot Format', 'Ad Slot Floor Price', 'Creative ID', 'Bidding Price',
       'Paying Price', 'Landing Page URL', 'Advertiser ID',
       'User Profile IDs'],
      dtype='object')
Index(['Bid ID', 'Timestamp', 'Log Type', 'iPinYou ID', 'User-Agent', 'IP',
       'Region ID', 'City ID', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL',
       'Ad Slot ID', 'Ad Slot Width', 'Ad Slot Heig

In [20]:
cols_to_drop=['iPinYou ID'
              ,'IP'
              ,'Domain'
              ,'URL'
              ,'Anonymous URL'
              ,'Ad Slot ID'
              ,'Creative ID']

In [21]:
for index, value in all_data.items(): 
    value.drop(cols_to_drop, axis=1, inplace =True)
    if index!='bids': 
        value.drop('Landing Page URL', axis=1, inplace=True)

## Sanity check before joining <a class='anchor' id="5.0.1"></a>

### Impressions

Let's see what columns differ when inner joining

In [22]:
test_bids_to_impressions=pd.merge(bids, impressions, on="Bid ID", suffixes=('_bid', '_imp'), how="inner")

In [23]:
test_bids_to_impressions.shape

(1821350, 29)

In [24]:
test_bids_to_impressions.head(1)

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID_bid,City ID_bid,Ad Exchange_bid,Ad Slot Width_bid,Ad Slot Height_bid,Ad Slot Visibility_bid,Ad Slot Format_bid,Ad Slot Floor Price_bid,Bidding Price_bid,Advertiser ID_bid,User Profile IDs_bid,Timestamp_imp,Log Type,User-Agent_imp,Region ID_imp,City ID_imp,Ad Exchange_imp,Ad Slot Width_imp,Ad Slot Height_imp,Ad Slot Visibility_imp,Ad Slot Format_imp,Ad Slot Floor Price_imp,Bidding Price_imp,Paying Price,Advertiser ID_imp,User Profile IDs_imp
0,7b6195de0d14203f92001da653bf1de,20130606000104009,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",216,217,2,468,60,1,0,5,300,3386,NaN,20130606000107102,1,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),216,217,2,468,60,1,0,5,300,18,3386,NaN


In [25]:
len_test=len(test_bids_to_impressions)
for column in bids.columns:
    if column!='Bid ID':
        right=column+'_imp'
        left=column+'_bid'
        n_diff=(test_bids_to_impressions[right]!=test_bids_to_impressions[left]).sum(axis=0)
        percentage=n_diff/len_test
        print('''for column %s 
            number : %s
            percentage: %s'''%(column, n_diff, percentage))
              
              

for column Timestamp 
            number : 1821350
            percentage: 1.0
for column User-Agent 
            number : 1160633
            percentage: 0.6372377631976281
for column Region ID 
            number : 89
            percentage: 4.886485299365855e-05
for column City ID 
            number : 89
            percentage: 4.886485299365855e-05
for column Ad Exchange 
            number : 0
            percentage: 0.0
for column Ad Slot Width 
            number : 0
            percentage: 0.0
for column Ad Slot Height 
            number : 0
            percentage: 0.0
for column Ad Slot Visibility 
            number : 0
            percentage: 0.0
for column Ad Slot Format 
            number : 0
            percentage: 0.0
for column Ad Slot Floor Price 
            number : 0
            percentage: 0.0
for column Bidding Price 
            number : 0
            percentage: 0.0
for column Advertiser ID 
            number : 0
            percentage: 0.0
for column User P

Conclusion: just keep User Profile ID + TimeStamp + User Agent from impression

### Bids to clicks

In [26]:
test_bids_to_clicks=pd.merge(impressions, clicks, on="Bid ID", suffixes=('_bid', '_clk'), how="inner")

In [27]:
test_bids_to_clicks.shape

(1290, 31)

In [28]:
test_bids_to_clicks.head(1)

,Bid ID,Timestamp_bid,Log Type_bid,User-Agent_bid,Region ID_bid,City ID_bid,Ad Exchange_bid,Ad Slot Width_bid,Ad Slot Height_bid,Ad Slot Visibility_bid,Ad Slot Format_bid,Ad Slot Floor Price_bid,Bidding Price_bid,Paying Price_bid,Advertiser ID_bid,User Profile IDs_bid,Timestamp_clk,Log Type_clk,User-Agent_clk,Region ID_clk,City ID_clk,Ad Exchange_clk,Ad Slot Width_clk,Ad Slot Height_clk,Ad Slot Visibility_clk,Ad Slot Format_clk,Ad Slot Floor Price_clk,Bidding Price_clk,Paying Price_clk,Advertiser ID_clk,User Profile IDs_clk
0,617e491a10952bf3aa7f34ed1116a4fe,20130606000106798,1,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17 SE 2.X MetaSr 1.0",1,1,2,250,250,2,0,5,238,100,3427,NaN,20130606000239531,2,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17 SE 2.X MetaSr 1.0",1,1,2,250,250,2,0,5,238,0,3427,NaN


In [29]:
len_test=len(test_bids_to_clicks)
for column in bids.columns:
    if column!='Bid ID':
        right=column+'_clk'
        left=column+'_bid'
        n_diff=(test_bids_to_clicks[right]!=test_bids_to_clicks[left]).sum(axis=0)
        percentage=n_diff/len_test
        print('''for column %s 
            number : %s
            percentage: %s'''%(column, n_diff, percentage))
              
              

for column Timestamp 
            number : 1288
            percentage: 0.9984496124031008
for column User-Agent 
            number : 110
            percentage: 0.08527131782945736
for column Region ID 
            number : 0
            percentage: 0.0
for column City ID 
            number : 0
            percentage: 0.0
for column Ad Exchange 
            number : 0
            percentage: 0.0
for column Ad Slot Width 
            number : 0
            percentage: 0.0
for column Ad Slot Height 
            number : 0
            percentage: 0.0
for column Ad Slot Visibility 
            number : 0
            percentage: 0.0
for column Ad Slot Format 
            number : 0
            percentage: 0.0
for column Ad Slot Floor Price 
            number : 0
            percentage: 0.0
for column Bidding Price 
            number : 0
            percentage: 0.0
for column Advertiser ID 
            number : 0
            percentage: 0.0
for column User Profile IDs 
            numbe

### Imp to click 

In [30]:
test_imp_to_clicks=pd.merge(impressions, clicks, on="Bid ID", suffixes=('_imp', '_clk'), how="inner")

In [31]:
test_imp_to_clicks.shape

(1290, 31)

In [32]:
test_imp_to_clicks.head(1)

,Bid ID,Timestamp_imp,Log Type_imp,User-Agent_imp,Region ID_imp,City ID_imp,Ad Exchange_imp,Ad Slot Width_imp,Ad Slot Height_imp,Ad Slot Visibility_imp,Ad Slot Format_imp,Ad Slot Floor Price_imp,Bidding Price_imp,Paying Price_imp,Advertiser ID_imp,User Profile IDs_imp,Timestamp_clk,Log Type_clk,User-Agent_clk,Region ID_clk,City ID_clk,Ad Exchange_clk,Ad Slot Width_clk,Ad Slot Height_clk,Ad Slot Visibility_clk,Ad Slot Format_clk,Ad Slot Floor Price_clk,Bidding Price_clk,Paying Price_clk,Advertiser ID_clk,User Profile IDs_clk
0,617e491a10952bf3aa7f34ed1116a4fe,20130606000106798,1,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17 SE 2.X MetaSr 1.0",1,1,2,250,250,2,0,5,238,100,3427,NaN,20130606000239531,2,"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17 SE 2.X MetaSr 1.0",1,1,2,250,250,2,0,5,238,0,3427,NaN


In [33]:
len_test=len(test_imp_to_clicks)
for column in impressions.columns:
    if column!='Bid ID':
        left=column+'_imp'
        right=column+'_clk'
        n_diff=(test_imp_to_clicks[right]!=test_imp_to_clicks[left]).sum(axis=0)
        percentage=n_diff/len_test
        print('''for column %s 
            number : %s
            percentage: %s'''%(column, n_diff, percentage))
              
              

for column Timestamp 
            number : 1288
            percentage: 0.9984496124031008
for column Log Type 
            number : 1290
            percentage: 1.0
for column User-Agent 
            number : 110
            percentage: 0.08527131782945736
for column Region ID 
            number : 0
            percentage: 0.0
for column City ID 
            number : 0
            percentage: 0.0
for column Ad Exchange 
            number : 0
            percentage: 0.0
for column Ad Slot Width 
            number : 0
            percentage: 0.0
for column Ad Slot Height 
            number : 0
            percentage: 0.0
for column Ad Slot Visibility 
            number : 0
            percentage: 0.0
for column Ad Slot Format 
            number : 0
            percentage: 0.0
for column Ad Slot Floor Price 
            number : 0
            percentage: 0.0
for column Bidding Price 
            number : 0
            percentage: 0.0
for column Paying Price 
            number : 12

In [34]:
test_imp_to_clicks[['User Profile IDs_imp','User Profile IDs_clk']].head()

,User Profile IDs_imp,User Profile IDs_clk
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,"10063,10052,11379,11092,13866,10083,10684,11423,14273,10111","10063,10052,11379,11092,13866,10083,10684,11423,14273,10111"


In [35]:
test_imp_to_clicks[['Log Type_imp','Log Type_clk']].head()

,Log Type_imp,Log Type_clk
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2


In [36]:
test_imp_to_clicks[['Paying Price_imp','Paying Price_clk']].head()

,Paying Price_imp,Paying Price_clk
0,100,0
1,71,0
2,113,0
3,113,0
4,89,0


In [58]:
test_imp_to_clicks['Paying Price_clk'].sum()

0

conclusion keep only Timestamp, User-Agent user profile IDs AND REGION ID TO GET BOOLEAN TO WHETHER IT WAS CLICKED

### get number of impressions/clicks based on ID 

In [68]:
test_count=impressions.groupby(['Bid ID']).size()

In [70]:
type(test_count)

pandas.core.series.Series

In [77]:
test_merge_imp=pd.merge(impressions[['Bid ID'
           , 'Timestamp'
           , 'User-Agent'
           , 'Log Type'
           , 'Paying Price'
           , 'User Profile IDs']].drop_duplicates('Bid ID'), test_count.rename('Count Impression'), left_on='Bid ID',right_index=True)

In [80]:
test_merge_imp.head()

,Bid ID,Timestamp,User-Agent,Log Type,Paying Price,User Profile IDs,Count Impression
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",1,207,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",1
1,5bd0cbeb2f82fb94e56b7dc2e6b77ec,20130606000104252,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; .NET4.0C; .NET CLR 1.1.4322; SE 2.X MetaSr 1.0),1,72,"13800,10024",1
2,faf17eac9cabf1be598f4e75f40d501d,20130606000104253,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1,108,"10059,13866,10063,10111",1
3,234870d3864ad1852fe04b172f340be3,20130606000104308,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1,81,"10057,10063,10024,13800,13866,10110",1
4,20fb85784972aebed3dda8d20ca87afb,20130606000104329,Opera/9.80 (Android; Opera Mini/7.7.33548/29.3551; U; zh) Presto/2.8.119 Version/11.10,1,89,"13866,10111",1


In [83]:
sum(test_merge_imp['Count Impression'])-len(impressions)

0

## Join all <a class='anchor' id="5.1"></a>

In [108]:
impression_count=impressions.groupby(['Bid ID']).size()

impression_to_merge =pd.merge(impressions[['Bid ID'
           , 'Timestamp'
           , 'User-Agent'
           , 'Log Type'
           , 'Paying Price'
           , 'User Profile IDs']].drop_duplicates('Bid ID'), test_count.rename('n_impressions'), left_on='Bid ID',right_index=True)


df=pd.merge(bids, impression_to_merge, on="Bid ID", suffixes=('_bid', '_imp'), how="left")

clicks_count=clicks.groupby(['Bid ID']).size()

clicks_to_merge =pd.merge(clicks[['Bid ID'
                       , 'Timestamp'
                      , 'User-Agent'
                      , 'User Profile IDs'
                       , 'Region ID'
                      ]].drop_duplicates('Bid ID'), clicks_count.rename('n_clicks'), left_on='Bid ID',right_index=True)
           
df=pd.merge(df, clicks_to_merge, left_on="Bid ID",right_on="Bid ID", suffixes=('_bids_join_impressions', '_clicks'), how="left")

In [109]:
df.shape

(9586949, 25)

In [110]:
bids.shape

(9586949, 14)

In [112]:
print(df['n_impressions'].sum())
print(impressions.shape)
print(df['n_clicks'].sum())
print(clicks.shape)

1821350.0
(1821350, 16)
1289.0
(1289, 16)


In [113]:
df.head(5)

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,n_impressions,Timestamp,User-Agent,User Profile IDs,Region ID_clicks,n_clicks
0,b382c1c156dcbbd5b9317cb50f6a747b,20130606000104008,mozilla/4.0 (compatible; msie 6.0; windows nt 5.1; sv1; qqdownload 718),80,87,1,300,250,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7b6195de0d14203f92001da653bf1de,20130606000104009,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",216,217,2,468,60,1,0,5,300,3386,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),1.0,18.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2ea9fe21cf7350fcb5696d8cff0bbeaa,20130606000104012,mozilla/4.0 (compatible; msie 8.0; windows nt 5.1; trident/4.0; .net clr 2.0.50727),65,69,1,950,90,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104014,"mozilla/5.0 (windows nt 5.1) applewebkit/537.17 (khtml, like gecko) chrome/24.0.1312.57 safari/537.17",106,117,1,950,90,0,1,0,227,3427,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",1.0,207.0,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",1.0,NaN,NaN,NaN,NaN,NaN
4,faf17eac9cabf1be598f4e75f40d501d,20130606000104016,"mozilla/5.0 (windows nt 5.1) applewebkit/537.1 (khtml, like gecko) chrome/21.0.1180.89 safari/537.1",94,95,1,950,90,0,1,0,227,3358,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1.0,108.0,"10059,13866,10063,10111",1.0,NaN,NaN,NaN,NaN,NaN


In [137]:
df['has impression']=~(df['Timestamp_imp'].isna())
df['has click']=~(df['Region ID_clicks'].isna())

In [139]:
df['has impression']=df['has impression'].astype(int)
df['has click']=df['has click'].astype(int)

In [140]:
print(df['has impression'].sum())
print(df['has click'].sum())

1815075
1159


In [141]:
df.head(5)

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,n_impressions,Timestamp,User-Agent,User Profile IDs,Region ID_clicks,n_clicks,has impression,has click
0,b382c1c156dcbbd5b9317cb50f6a747b,20130606000104008,mozilla/4.0 (compatible; msie 6.0; windows nt 5.1; sv1; qqdownload 718),80,87,1,300,250,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,7b6195de0d14203f92001da653bf1de,20130606000104009,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",216,217,2,468,60,1,0,5,300,3386,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),1.0,18.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,0
2,2ea9fe21cf7350fcb5696d8cff0bbeaa,20130606000104012,mozilla/4.0 (compatible; msie 8.0; windows nt 5.1; trident/4.0; .net clr 2.0.50727),65,69,1,950,90,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104014,"mozilla/5.0 (windows nt 5.1) applewebkit/537.17 (khtml, like gecko) chrome/24.0.1312.57 safari/537.17",106,117,1,950,90,0,1,0,227,3427,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",1.0,207.0,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",1.0,NaN,NaN,NaN,NaN,NaN,1,0
4,faf17eac9cabf1be598f4e75f40d501d,20130606000104016,"mozilla/5.0 (windows nt 5.1) applewebkit/537.1 (khtml, like gecko) chrome/21.0.1180.89 safari/537.1",94,95,1,950,90,0,1,0,227,3358,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1.0,108.0,"10059,13866,10063,10111",1.0,NaN,NaN,NaN,NaN,NaN,1,0


In [121]:
print(len(clicks))
print(len(impressions))
print(df['has impression'].sum()/len(impressions))
print(df['has click'].sum()/len(clicks))

1289
1821350
0.996554753342301
0.8991466252909232


In [91]:
user_profile_cols = [col for col in df.columns if 'ID' in col]
user_profile_cols

['Bid ID',
 'Region ID',
 'City ID',
 'Advertiser ID',
 'User Profile IDs_bid',
 'User Profile IDs_imp']

## Reduce size

In [92]:
df.head()

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,has impression
0,b382c1c156dcbbd5b9317cb50f6a747b,20130606000104008,mozilla/4.0 (compatible; msie 6.0; windows nt 5.1; sv1; qqdownload 718),80,87,1,300,250,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,False
1,7b6195de0d14203f92001da653bf1de,20130606000104009,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",216,217,2,468,60,1,0,5,300,3386,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),1.0,18.0,NaN,True
2,2ea9fe21cf7350fcb5696d8cff0bbeaa,20130606000104012,mozilla/4.0 (compatible; msie 8.0; windows nt 5.1; trident/4.0; .net clr 2.0.50727),65,69,1,950,90,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,False
3,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104014,"mozilla/5.0 (windows nt 5.1) applewebkit/537.17 (khtml, like gecko) chrome/24.0.1312.57 safari/537.17",106,117,1,950,90,0,1,0,227,3427,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",1.0,207.0,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",True
4,faf17eac9cabf1be598f4e75f40d501d,20130606000104016,"mozilla/5.0 (windows nt 5.1) applewebkit/537.1 (khtml, like gecko) chrome/21.0.1180.89 safari/537.1",94,95,1,950,90,0,1,0,227,3358,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1.0,108.0,"10059,13866,10063,10111",True


In [93]:
cols_to_drop = ['Bid ID'
                , 'iPinYou ID_bid'
                , 'Domain_bid'
               ]

## Sample the dataset <a class='anchor' id="5.2"></a>

The data is a bit too big (too much time to solve) so I'll reduce the size to merge a first full dataset

In [95]:
df=df.sample(n=100000, random_state=100)

In [96]:
print(df.shape)

(100000, 20)


In [124]:
df.head(1)

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,has impression
7969425,55bdd298c4a31cc89e2148913d9fd88e,20130606202116961,"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729),gzip(gfe),gzip(gfe)",276,291,2,336,280,2,0,5,300,1458,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729),1.0,50.0,NaN,1


In [123]:
df['has impression']=df['has impression'].astype(int)

In [124]:
df.head(5)

,Bid ID,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,n_impressions,Timestamp,User-Agent,User Profile IDs,Region ID_clicks,n_clicks,has impression,has click
0,b382c1c156dcbbd5b9317cb50f6a747b,20130606000104008,mozilla/4.0 (compatible; msie 6.0; windows nt 5.1; sv1; qqdownload 718),80,87,1,300,250,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
1,7b6195de0d14203f92001da653bf1de,20130606000104009,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),gzip(gfe),gzip(gfe)",216,217,2,468,60,1,0,5,300,3386,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0),1.0,18.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,False
2,2ea9fe21cf7350fcb5696d8cff0bbeaa,20130606000104012,mozilla/4.0 (compatible; msie 8.0; windows nt 5.1; trident/4.0; .net clr 2.0.50727),65,69,1,950,90,1,1,0,227,3427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False
3,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104014,"mozilla/5.0 (windows nt 5.1) applewebkit/537.17 (khtml, like gecko) chrome/24.0.1312.57 safari/537.17",106,117,1,950,90,0,1,0,227,3427,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.57 Safari/537.17",1.0,207.0,"10063,10684,10083,13403,10059,10024,10048,10057,13678,10079,10006,10076,10075,13866,10052,10093,14273,10110",1.0,NaN,NaN,NaN,NaN,NaN,1,False
4,faf17eac9cabf1be598f4e75f40d501d,20130606000104016,"mozilla/5.0 (windows nt 5.1) applewebkit/537.1 (khtml, like gecko) chrome/21.0.1180.89 safari/537.1",94,95,1,950,90,0,1,0,227,3358,NaN,2.013061e+16,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",1.0,108.0,"10059,13866,10063,10111",1.0,NaN,NaN,NaN,NaN,NaN,1,False


# Modeling 

In [161]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [136]:
x = df[['Region ID'
        ,'City ID'
        ,'Ad Exchange'
        ,'Ad Slot Width'
        , 'Ad Slot Height'
        , 'Ad Slot Visibility'
        , 'Ad Slot Format'
        , 'Ad Slot Floor Price'
        , 'Bidding Price'
        , 'Advertiser ID']]
y = df['has impression']

In [137]:
y.head(1)

7969425    1
Name: has impression, dtype: int64

In [138]:
x.head(1)

,Region ID,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID
7969425,276,291,2,336,280,2,0,5,300,1458


In [142]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(80000, 10)
(20000, 10)
(80000,)
(20000,)


In [146]:
LogisticRegressionModel = linear_model.LogisticRegression(solver = 'newton-cg', multi_class='multinomial')

In [147]:
LogisticRegressionModel.fit(x_train, y_train)

/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [152]:
'the logistic regression accuracy is: %s'%(LogisticRegressionModel.score(x_train,y_train))

'the logistic regression accuracy is: 0.83945'

In [162]:
models= {
    'Logistic': LogisticRegression(solver = 'newton-cg', multi_class='multinomial')
    ,'KNN': KNeighborsClassifier(3)
    , 'SVC1':SVC(kernel="linear", C=0.025)
    , 'SVC2': SVC(gamma=2, C=1)
    , 'GPC': GaussianProcessClassifier(1.0 * RBF(1.0))
    , 'DTX': DecisionTreeClassifier(max_depth=5)
    , 'RF': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    , 'MLP': MLPClassifier(alpha=1, max_iter=1000)
    ,  'AdaBoost': AdaBoostClassifier()
    , 'Gaussian NB':GaussianNB()
    , 'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis()
    
}

In [ ]:
scores= {}
for model, clf in models.items():
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    scores[model]=score

/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/Armand/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


In [170]:
scores

{}

In [175]:
for model, clf in models.items():
    scores[model]=1

In [176]:
scores

{'one': 1,
 'Logistic': 1,
 'KNN': 1,
 'SVC1': 1,
 'SVC2': 1,
 'GPC': 1,
 'DTX': 1,
 'RF': 1,
 'MLP': 1,
 'AdaBoost': 1,
 'Gaussian NB': 1,
 'QuadraticDiscriminantAnalysis': 1}

# Add User profile info <a class="anchor" id="6"></a>

In [98]:
for key, value in all_data.items(): 
    print('shape of %s is %s'%(key, value.shape))

shape of bids is (9586949, 14)
shape of impressions is (1821350, 16)
shape of clicks is (1289, 16)
shape of conversions is (43, 16)


In [99]:
mapping=pd.read_table('/Users/Armand/Capstone/ipinyou.contest.dataset/user.profile.tags.en.txt', names=('key','value'))

In [100]:
s=df['User Profile IDs_imp'].str.split(pat=',')
final_to_concatenate=pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

In [101]:
final_to_concatenate.head()
final_to_concatenate.fillna(0, inplace=True)

In [ ]:
merged = df.merge(final_to_concatenate, left_index=True, right_index=True, how='left')

In [ ]:
merged.head()

In [ ]:
merged.to_csv('final.csv')

In [195]:
def one_day_training_testing(date):
    # READ DATA FOR 1 DAY 
    unzipped_file = bz2.BZ2File('bid.'+date+'.txt.bz2', "r")
    bids=pd.read_table(unzipped_file, names=header_season2_bids)
    
    unzipped_file = bz2.BZ2File('imp.'+date+'.txt.bz2', "r")
    impressions=pd.read_table(unzipped_file, names=header_season2)
    
    unzipped_file = bz2.BZ2File('clk.'+date+'.txt.bz2', "r")
    clicks=pd.read_table(unzipped_file, names=header_season2)
    
    unzipped_file = bz2.BZ2File('conv.'+date+'.txt.bz2', "r")
    conversions=pd.read_table(unzipped_file, names=header_season2)

    all_data={'bids': bids, 'impressions':impressions, 'clicks':clicks, 'conversions':conversions}
    for key, value in all_data.items(): 
        print('shape of %s is %s'%(key, value.shape))

    #ASSUMPTION 1: SOME COLUMNS CAN BE DROPPED BECAUSE THEY HAVE NO PREDICTIVE POWER
    cols_to_drop=['iPinYou ID'
                ,'IP'
                ,'Domain'
                ,'URL'
                ,'Anonymous URL'
                ,'Ad Slot ID'
                ,'Creative ID']

    for index, value in all_data.items(): 
        value.drop(cols_to_drop, axis=1, inplace =True)
        if index!='bids': 
            value.drop('Landing Page URL', axis=1, inplace=True)

    # MERGE 
    ## ASSUMPTION 2.1:COUNT DUPLICATES ON BID ID FOR IMPRESSIONS AND ADD THEM TO LINE
    ## ASSUMPTION 2.2: ONLY KEEP COLUMNS THAT ACTUALLY DIFFER IN THE IMPRESSION TABLE


    impression_count=impressions.groupby(['Bid ID']).size()

    impression_to_merge =pd.merge(impressions[['Bid ID'
            , 'Timestamp'
            , 'User-Agent'
            , 'Log Type'
            , 'Paying Price'
            , 'User Profile IDs']].drop_duplicates('Bid ID'), test_count.rename('n_impressions'), left_on='Bid ID',right_index=True)


    df=pd.merge(bids, impression_to_merge, on="Bid ID", suffixes=('_bid', '_imp'), how="left")

    ## ASSUMPTION 3.1: COUNT DUPLICATES FOR BID ID FOR CLICKS AND ADD THEM  
    ## ASSUMPTION 3.2: ONLY KEEP COLUMNS THAT ACTUALLY DIFFER IN THE CLICKS TABLE

    clicks_count=clicks.groupby(['Bid ID']).size()

    clicks_to_merge =pd.merge(clicks[['Bid ID'
                        , 'Timestamp'
                        , 'User-Agent'
                        , 'User Profile IDs'
                        , 'Region ID'
                        ]].drop_duplicates('Bid ID'), clicks_count.rename('n_clicks'), left_on='Bid ID',right_index=True)
            
    df=pd.merge(df, clicks_to_merge, left_on="Bid ID",right_on="Bid ID", suffixes=('_bids_join_impressions', '_clicks'), how="left")

    # ADD COLUMNS (QUITE REDUNDANT WITH n_clicks & n_impressions)
    df['has impression']=~(df['Timestamp_imp'].isna())
    df['has click']=~(df['Region ID_clicks'].isna())
    #CONVERT BOOL TO INT 
    df['has impression']=df['has impression'].astype(int)
    df['has click']=df['has click'].astype(int)

    #REDUCE SIZE   
    cols_to_drop = ['Bid ID']
    df.drop(cols_to_drop, axis=1, inplace =True)
    #SAMPLE THE DATASET
    df=df.sample(n=100000, random_state=100)
    return(df)

In [198]:
df_final=one_day_training_testing('20130606')
for date in dates_list[1:]:    
    df=one_day_training_testing(date)
    print('shape of df is %s'%str(df.shape))
    df_final=df_final.append(df)
    print('shape of df_final is %s'%str(df_final.shape))

shape of bids is (9586949, 21)
shape of impressions is (1821350, 24)
shape of clicks is (1289, 24)
shape of conversions is (43, 24)
shape of bids is (11132555, 21)
shape of impressions is (1805953, 24)
shape of clicks is (1158, 24)
shape of conversions is (50, 24)
shape of df is (100000, 26)
shape of df_final is (200000, 26)
shape of bids is (5226937, 21)
shape of impressions is (1634830, 24)
shape of clicks is (1302, 24)
shape of conversions is (52, 24)
shape of df is (100000, 26)
shape of df_final is (300000, 26)
shape of bids is (11880893, 21)
shape of impressions is (1651524, 24)
shape of clicks is (1250, 24)
shape of conversions is (32, 24)
shape of df is (100000, 26)
shape of df_final is (400000, 26)
shape of bids is (5610729, 21)
shape of impressions is (1920370, 24)
shape of clicks is (1779, 24)
shape of conversions is (150, 24)
shape of df is (100000, 26)
shape of df_final is (500000, 26)
shape of bids is (5094845, 21)
shape of impressions is (1745722, 24)
shape of clicks is (

In [199]:
df.shape

(100000, 26)

In [200]:
df_final.head(1)

,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,n_impressions,Timestamp,User-Agent,User Profile IDs,Region ID_clicks,n_clicks,has impression,has click
612529,20130606000849005,"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1,gzip(gfe),gzip(gfe)",124,129,2,728,90,2,0,5,300,1458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [201]:
df_final.shape

(700000, 26)

In [193]:
df_final.shape

(100000, 27)

In [207]:
s=df_final['User Profile IDs_imp'].str.split(pat=',')
dummies_to_concatenate=pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

df_dummified = df_final.merge(dummies_to_concatenate, left_index=True, right_index=True, how='left')

In [208]:
df_dummified.shape

(700000, 69)

In [209]:
df_dummified.head(1)

,Timestamp_bid,User-Agent_bid,Region ID_bids_join_impressions,City ID,Ad Exchange,Ad Slot Width,Ad Slot Height,Ad Slot Visibility,Ad Slot Format,Ad Slot Floor Price,Bidding Price,Advertiser ID,User Profile IDs_bid,Timestamp_imp,User-Agent_imp,Log Type,Paying Price,User Profile IDs_imp,n_impressions,Timestamp,User-Agent,User Profile IDs,Region ID_clicks,n_clicks,has impression,has click,10006,10024,10031,10048,10052,10057,10059,10063,10067,10074,10075,10076,10077,10079,10083,10093,10102,10110,10111,10684,11092,11278,11379,11423,11512,11576,11632,11680,11724,11944,13042,13403,13496,13678,13776,13800,13866,13874,14273,16593,16617,16661,16706
53,20130606000104118,"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0),gzip(gfe),gzip(gfe)",80,85,2,336,280,2,0,5,300,3386,NaN,2.013061e+16,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0),1.0,5.0,"10063,10059,13776,10111",1.0,NaN,NaN,NaN,NaN,NaN,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [206]:
dummies_to_concatenate

""
